## Note
⚠⚠ *This API is in early-stage development. Structure, naming conventions, and features may change without notice!* ⚠⚠

In [1]:
import circuitree as ct

## Installation
If you have not already followed the installation instructions, please do this first.

# Quick start

`circuitree` is a package for designing network topologies (*circuits*) by executing a tree search. Circuits are arranged in a tree by considering each circuit to be assembled in discrete steps (called *actions*). Each sequence of actions produces a circuit, and together, the union of all possible assembly sequences forms a searchable decision tree.

The class `CircuiTree` is an abstract class that contains the machinery for executing the search. To use `CircuiTree`, create a subclass that specifies the rules for assembling and representing circuits. An example of the intended usage is below.

### Designing a simple (bio)molecular circuit

First, we need to define a space of circuits that could be assembled and define some rules for assembly. 

Let's consider a design space containing two molecular components `A` and `B` that can interact with each other or themselves in an inhibitory or activating manner.

In [2]:
components = ["A", "B"]
interactions = ["activates", "inhibits"]

We need to also choose a way to represent each circuit uniquely with a string, called the `state` (or `state`). At minimum, the string should encode (i) which components are present, (ii) which interactions are present, and (iii) whether or not the assembly has terminated. It may also ensure that each unique circuit gets a unique string. For example, let's use the following representation:
* Each component is an uppercase letter (`A` and `B`) 
* Each type of interaction is a lowercase letter (`a` and `i` for activation and inhibition, respectively)
* Terminated (fully assembled) circuits begin with an asterisk `*`
* Each interaction has a 3-character code. `ABi` can be read "`A` inhibits `B`"
* Interactions are separated by an underscore `_`
* Components are separated from interactions by colons `::`
* Components and interactions are sorted in alphabetic order

For instance, the state string `*AB::ABi_BAi` denotes a terminally assembled circuit with components `A` and `B` that inhibit each other (i.e. a toggle switch). Sorting the interactions ensures that assembly steps are associative and order doesn't matter (`ABi` then `BAi` yields the same string as `BAi` then `ABi`). Also note that this representation does not account for symmetry, so if we assemble two circuits that are identical but the labels `A` and `B` are switched, they will have different state strings.

Recall that `CircuiTree` is an abstract class, which means we have to create a subclass before we can use it. Let's look at how we can encode this system for representing our circuits into our own subclass of `CircuiTree`.

In [6]:
from circuitree import CircuiTree
print(f"To use the CircuiTree class, define the following methods:")
print(*[f"\t{m}" for m in CircuiTree.__abstractmethods__], sep="\n")

To use the CircuiTree class, define the following methods:
	get_unique_state
	get_reward
	is_terminal
	do_action
	get_actions


We can easily define `is_terminal` and `get_unique_state` based on our rules for string representation.

In [ ]:
class MyTree(CircuiTree):
    def __init__(self, components, interactions, root, **kwargs):
        super().__init__(**kwargs)
        self.components = components
        self.interactions = interactions
        self.root = root

    def is_terminal(self, state):
        """Return True if the state is a terminal state."""
        return state.startswith("*")

    def get_unique_state(self, state: str) -> str:
        """Return a unique state name."""
        components, interactions = state.split("::")
        if components.startswith("*"):
            prefix = "*"
            components = components[1:]
        else:
            prefix = ""
        unique_components = "".join(sorted(components))
        unique_interactions = "_".join(sorted(interactions.split("_")))
        return prefix + unique_components + "::" + unique_interactions

The methods `do_action` and `get_actions`  When assembling a circuit, we consider the assembly to be a sequence of `state`s and `action`s. 
* `state`: A stage in circuit assembly. Can be terminal or non-terminal (intermediate). Represented by a string.
* `action`: Transforms one state into the next. Non-terminal states may have multiple possible actions available. Terminal states have no available actions by definition.

We arrange these in a tree**, where nodes of the graph are `state`s and edges are `state`-`action` pairs that transform the `state` into another `state`. The `root` state of the tree is the common starting point for all assemblies. For the two-component case, that could be a non-terminal circuit with components `A` and `B` but no interactions - `AB::`. Assembling a circuit is therefore equivalent to traversing the tree from its root to a leaf.

The function `get_actions` returns the possible actions that can be taken from a state, and the function `do_action` returns the new state that results from applying an action to a given state.

** Technically, this is a rooted directed acyclic graph (DAG) because the order of assemblies doesn't matter, multiple edges can coincide on the same node. Nonetheless we use "tree" for simplicity.

In [ ]:

    def get_actions(self, state: str):
        if self.is_terminal(state):
            return list()

        # Terminating assembly is always an option
        actions = ["*terminate*"]

        components, interactions_joined = state.strip("*").split("::")
        interactions = set(ixn[:2] for ixn in interactions_joined.split("_"))
        for action_group in self.edge_options:
            if action_group:
                c1, c2, _ = action_group[0]
                if (
                    (c1 in components)
                    and (c2 in components)
                    and (c1 + c2) not in interactions
                ):
                    for action in action_group:
                        actions.append(action)

        return actions

    def do_action(self, state: str, action: str) -> str:
        if action == "*terminate*":
            new_state = "*" + state
        else:
            # Root node
            if state == ".":
                components = list()
                interactions = list()
            else:
                components, interactions = state.split("::")
            if len(action) == 1:
                new_state = "".join([components, action, "::", interactions])
            elif len(action) == 3:
                delim = ("", "_")[bool(interactions)]
                new_state = "::".join(
                    [components, delim.join([interactions, action])]
                )
        return new_state
